Drop the DAW – Sound Design in Python - Isaac Roberts - ADC20  
https://www.youtube.com/watch?v=Q40qEg8Yq5c&t=1323s  

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Audio

In [2]:
class Sig:
    def __init__(self, sec=1, const=0, array=None):
        self.sr = 44100
        if array is not None:
            self.array = array
        else:
            l = int(self.sr * sec)
            self.array = np.full(l, const)
    
    def sec(self):
        return len(self.array) / self.sr
    
    def size(self):
        return len(self.array)
            
    def is_sig(self):
        return True

    def make_sig(self, arg):
        # if sig is number, create Sig object"
        try:
            arg.is_sig()
            return arg
        except:
            sec = len(self.array) / self.sr
            return Sig(sec=sec, const=arg)
        
    def integrate(self):
        a = np.copy(self.array)
        a = a / self.sr
        a = np.cumsum(a)
        return Sig(array=a)
        
    def plot(self, start_sec=0, end_sec=None):
        if end_sec is None:
            end_sec = self.sec()
        n1 = int(self.sr * start_sec)
        n2 = int(self.sr * end_sec)
        a = self.array[n1:n2]
        plt.plot(a)
        
    def play(self, start_sec=0, end_sec=None):
        if end_sec is None:
            end_sec = self.sec()
        n1 = int(self.sr * start_sec)
        n2 = int(self.sr * end_sec)
        a = self.array[n1:n2]   
        return Audio(a, rate=self.sr, autoplay=True, normalize=False)

    def const(self, c):
        sig = Sig(self.sec, const=c)
        return sig
    
    def __add__(self, sig):
        a = self.array + sig.array
        return Sig(array=a)
    
    def __mul__(self, sig):
        a = self.array * sig.array
        return Sig(array=a)
    
    def range(self, min, max):
        x0 = np.min(self.array)
        x1 = np.max(self.array)
        m = (max - min) / (x1 -x0)
        n = min - m * x0
        a = np.copy(self.array)
        a = m * a + n
        return Sig(array=a)
    
    def sin(self, freq):
        freq = self.make_sig(freq)
        s = freq.integrate().array
        a = np.sin(2 * np.pi * s)
        return Sig(array=a)
                          
    def saw(self, freq):      
        freq = self.make_sig(freq)
        s = freq.integrate().array
        a = ((2 * s) % 2) - 1
        return Sig(array=a)
    
    def rect(self, freq):
        freq = self.make_sig(freq)
        s = freq.integrate().array
        a = np.where( 2 * s % 2 > 1, -1, 1)
        return Sig(array=a)

    def tri(self, freq):
        freq = self.make_sig(freq)
        s = freq.integrate().array
        a = np.abs((1 * s - 0.5) % 2 -1) * 2 -1
        return Sig(array=a)
    
    def asr(self, atk, sus, rel):
        atk_ar = np.linspace(0, 1, int(self.sr * atk))
        sus_ar = np.linspace(1, 1, int(self.sr * sus))
        rel_ar = np.linspace(1, 0, int(self.sr * rel))
        
        env = atk_ar
        env = np.append(env, sus_ar)
        env = np.append(env, rel_ar)
        
        if len(env) > len(self.array):
            env = env[0:len(self.array)]
            
        if len(env) < len(self.array):
            n = len(self.array) -len(env)
            env = np.append(env, np.zeros(n))

        return Sig(array=env)
    
    
    def mul(self, factor):
        factor = self.make_sig(factor)
        a = self.array * factor.array
        return Sig(array=a)
        

In [92]:
def instr(freq):
    env = Sig(1).asr(0.01, 0, 0.9)
    sig = Sig(1).rect(freq).mul(env).mul(0.1)
    return sig

In [123]:
def seq(instr, freqs, dur=1):
    
    # calc length of result using test tone
    test = instr(440)
    l = len(freqs) * dur * test.sr + test.size()
    
    
    a = np.zeros(int(l))
    os = 0
    for freq in freqs:
        tone = instr(freq)
        s = tone.size()
        a[os:s+os] = a[os:s+os] + tone.array[0:s]
        os = os + int(44100 * dur)
    return Sig(array=a)

In [125]:
sig = seq(instr, freqs=[440,550,660,770,880] * 5, dur=0.1)
sig.play()